In [1]:
!pip show tensorflow tensorflow-probability
!pip install tensorflow-probability==0.24.0
!pip install --upgrade keras-tuner
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git
!pip install mne #The MNE Package is installed
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip -y #Package with useful functions for motor imagery classification based in EEG.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!dir

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras
---
Name: tensorflow-probability
Version: 0.25.0
Summary: Probabilistic modeling and statistical inference in TensorFlow
Home-page: http://github.com/tensorflow/probability
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, cloudpickle, decorator, dm-tree, gast, numpy, six
Required-by: dopamine_r

In [2]:
import numpy as np
import os
import itertools
import random
import pickle
import mne
import h5py
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import keras_tuner as kt


from gcpds.databases.BCI_Competition_IV import Dataset_2a
from typing import Sequence, Tuple
from scipy.signal import iirnotch, filtfilt, butter, freqz
import matplotlib.pyplot as plt
from scipy.spatial import distance
import networkx as nx
from tqdm import tqdm
from mne.preprocessing import compute_current_source_density
from mne.channels import make_standard_montage, read_custom_montage
from scipy.signal import butter, filtfilt, resample, iirnotch
from gcpds.visualizations.series import plot_eeg
from scipy.stats import norm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from tqdm import tqdm

from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from gcpds.databases import GIGA_MI_ME
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import LeavePGroupsOut, StratifiedGroupKFold
from tensorflow.keras.models import Model
from scipy.spatial.distance import cdist
from sklearn.model_selection import GroupKFold
from tensorflow.keras.models import load_model

from keras_tuner import Objective
from keras_tuner import HyperModel
from keras.layers import Layer, Activation
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters

2025-06-16 00:04:19.336398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750032259.807308      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750032259.931499      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Carga los datos EEG para un sujeto específico con preprocesamiento, incluyendo filtrado y laplaciano superficial.
    Se recorta el tiempo entre los segundos 2 y 4.
    
    Args:
        db (Dataset_2a): Objeto del dataset.
        sbj (int): Identificador del sujeto (1-9).
        mode (str): 'training' o 'testing'.
        fs (float): Frecuencia de muestreo.

    Returns:
        tuple: Datos EEG preprocesados con laplaciano superficial y recortados en el tiempo (X), etiquetas (y).
    """
    # Cargar los datos del sujeto
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()  # Datos y etiquetas
    X = X[:, :-3, :]  # Seleccionar solo los canales EEG (22 canales)
    X = X * 1e6  # Convertir a microvoltios

    # Aplicar filtro Notch (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Aplicar filtro pasa banda (0.5 - 100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Recortar los datos entre el segundo 2 y el segundo 4
    start_sample = int(2 * fs)  # Inicio en el segundo 2
    end_sample = int(4 * fs)    # Fin en el segundo 4
    X = X[:, :, start_sample:end_sample]

    # Lista de nombres de los 22 canales EEG (sin EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Crear información para los canales EEG
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  # Usar la frecuencia original
        ch_types=["eeg"] * len(eeg_channel_names)  # Todos los canales son EEG
    )

    # Cargar un montaje estándar basado en el sistema 10/20
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Aplicar el cálculo del laplaciano superficial 
    laplacian_X = []
    for trial in X:
        # Crear un objeto RawArray para cada prueba
        raw = mne.io.RawArray(trial, info)
        # Calcular el laplaciano superficial
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Obtener los datos con el laplaciano aplicado
        laplacian_X.append(raw.get_data())

    # Reconvertir a un array numpy con la misma forma original
    X = np.stack(laplacian_X)

    return X, y

In [5]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Carga los datos EEG para un sujeto específico con preprocesamiento, incluyendo filtrado y laplaciano superficial.
    Se recorta el tiempo entre los segundos 2 y 4 y solo se incluyen las clases 'mano izquierda' (1) y 'mano derecha' (2).
    
    Args:
        db (Dataset_2a): Objeto del dataset.
        sbj (int): Identificador del sujeto (1-9).
        mode (str): 'training' o 'testing'.
        fs (float): Frecuencia de muestreo.

    Returns:
        tuple: Datos EEG preprocesados con laplaciano superficial y recortados en el tiempo (X), etiquetas (y).
    """
    # Cargar los datos del sujeto
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()  # Datos y etiquetas
    X = X[:, :-3, :]  # Seleccionar solo los canales EEG (22 canales)
    X = X * 1e6  # Convertir a microvoltios

    # Aplicar filtro Notch (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Aplicar filtro pasa banda (0.5 - 100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Recortar los datos entre el segundo 2 y el segundo 4
    start_sample = int(2 * fs)  # Inicio en el segundo 2
    end_sample = int(4 * fs)    # Fin en el segundo 4
    X = X[:, :, start_sample:end_sample]

    # Filtrar solo las clases de interés (1: mano izquierda, 2: mano derecha)
    clases = [0, 1]
    mask = np.isin(y, clases)
    X = X[mask]
    y = y[mask]

    # Lista de nombres de los 22 canales EEG (sin EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Crear información para los canales EEG
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  # Usar la frecuencia original
        ch_types=["eeg"] * len(eeg_channel_names)  # Todos los canales son EEG
    )

    # Cargar un montaje estándar basado en el sistema 10/20
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Aplicar el cálculo del laplaciano superficial 
    laplacian_X = []
    for trial in X:
        # Crear un objeto RawArray para cada prueba
        raw = mne.io.RawArray(trial, info)
        # Calcular el laplaciano superficial
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Obtener los datos con el laplaciano aplicado
        laplacian_X.append(raw.get_data())

    # Reconvertir a un array numpy con la misma forma original
    X = np.stack(laplacian_X)

    return X, y

In [14]:
# Crear instancia del dataset
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0 

# Cargar los datos del sujeto 9 en modo 'training'
X, y = load_BCICIV2a(db, sbj=2, mode='training', fs=fs)
print(f'tamaño de X:', X.shape)
print(f'tamaño de X:', y.shape)

tamaño de X: (142, 22, 500)
tamaño de X: (142,)


In [15]:
@register_keras_serializable(package="CustomLayers")
class TakensConv1D(tf.keras.layers.Layer):
    def __init__(self, dx=4, dy=3, tau=1, mu=4, **kwargs):
        super().__init__(**kwargs)
        self.dx = int(dx)
        self.dy = int(dy)
        self.tau = int(tau)
        self.mu = int(mu)
        self.num_filters = self.dx + self.dy + 1

    def build(self, input_shape):
        kernel_size = self.mu + (self.dx - 1) * self.tau + 1
        kernel_shape = (kernel_size, 1, self.num_filters)
        kernel = tf.zeros(kernel_shape, dtype=tf.float32)

        offsets_x = self.mu + tf.range(self.dx) * self.tau
        offsets_y = tf.range(1, self.dy + 1) * self.tau
        offset_y_t = tf.constant([0], dtype=tf.int32)

        filas_x = tf.range(self.dx)
        filas_y = self.dx + tf.range(self.dy)
        fila_y_t = tf.constant([self.dx + self.dy])

        cols_x = filas_x
        cols_y = filas_y
        col_y_t = fila_y_t

        idx_x = tf.stack([filas_x, offsets_x, cols_x], axis=1)
        idx_y = tf.stack([filas_y, offsets_y, cols_y], axis=1)
        idx_yt = tf.stack([fila_y_t, offset_y_t, col_y_t], axis=1)

        indices_total = tf.concat([idx_x, idx_y, idx_yt], axis=0)
        updates = tf.ones([tf.shape(indices_total)[0]], dtype=tf.float32)

        sort_order = tf.argsort(indices_total[:, 0])
        indices_sorted = tf.gather(indices_total, sort_order)

        row_for_scatter = tf.cast(indices_sorted[:, 1], tf.int32)
        col_for_scatter = tf.cast(indices_sorted[:, 2], tf.int32)

        final_indices = tf.stack([row_for_scatter,
                                  tf.zeros_like(row_for_scatter),
                                  col_for_scatter], axis=1)

        kernel = tf.scatter_nd(final_indices, updates, kernel_shape)
        self.kernel = kernel.numpy()[::-1]

        self.conv1d = tf.keras.layers.Conv1D(
            filters=self.num_filters,
            kernel_size=kernel_size,
            strides=self.tau,
            padding="valid",
            use_bias=False
        )
        self.conv1d.build((None, input_shape[2], 1))
        self.conv1d.set_weights([self.kernel])
        self.conv1d.trainable=False

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        channels = tf.shape(inputs)[1]
        time_steps = tf.shape(inputs)[2]

        reshaped = tf.reshape(inputs, (-1, time_steps, 1))
        conv_output = self.conv1d(reshaped)
        new_time = tf.shape(conv_output)[1]

        output = tf.reshape(conv_output, 
                            (batch_size, channels, new_time, self.num_filters))

        x_sub_t_minus_mu = output[..., :self.dx]
        y_sub_t_minus_1 = output[..., self.dx:self.dx + self.dy]
        y_sub_t = output[..., -1:]

        return x_sub_t_minus_mu, y_sub_t_minus_1, y_sub_t
       

@register_keras_serializable(package="CustomLayers")
class KernelLayer(tf.keras.layers.Layer):
    def __init__(self, 
                 amplitude=1.0,
                 trainable_amplitude=False, 
                 length_scale=1.0,
                 trainable_length_scale=True,
                 alpha=1.0,  # Solo usado para Rational Quadratic
                 trainable_alpha=True,
                 kernel_type="gaussian",  # "gaussian" o "rational_quadratic"
                 **kwargs):
        super(KernelLayer, self).__init__(**kwargs)

        self.init_amplitude = amplitude
        self.trainable_amplitude = trainable_amplitude
        self.init_length_scale = length_scale
        self.trainable_length_scale = trainable_length_scale
        self.init_alpha = alpha
        self.trainable_alpha = trainable_alpha
        self.kernel_type = kernel_type.lower()

    def build(self, input_shape):
        self.amplitude = self.add_weight(
            name="amplitude",
            shape=(),
            initializer=tf.constant_initializer(self.init_amplitude),
            trainable=self.trainable_amplitude,
            dtype=self.dtype
        )

        self.length_scale = self.add_weight(
            name="length_scale",
            shape=(),
            initializer=tf.constant_initializer(self.init_length_scale),
            trainable=self.trainable_length_scale,
            dtype=self.dtype
        )

        if self.kernel_type == "rational_quadratic":
            self.alpha = self.add_weight(
                name="alpha",
                shape=(),
                initializer=tf.constant_initializer(self.init_alpha),
                trainable=self.trainable_alpha,
                dtype=self.dtype
            )

        super(KernelLayer, self).build(input_shape)

    def call(self, X):
        if self.kernel_type == "gaussian":
            kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale
            )
        elif self.kernel_type == "rational_quadratic":
            kernel = tfp.math.psd_kernels.RationalQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale,
                scale_mixture_rate=self.alpha
            )
        else:
            raise ValueError(f"Unsupported kernel_type: {self.kernel_type}")
        
        return kernel.matrix(X, X)
       
@register_keras_serializable(package="CustomLayers")
class TransferEntropyLayer(tf.keras.layers.Layer):
    def __init__(self, alpha=2, **kwargs):

        super().__init__(**kwargs)
        self.alpha = int(alpha)

    def compute_entropy(self, K_hadamard):
        
        trace_hadamard = tf.reduce_sum(tf.linalg.diag_part(K_hadamard), axis=-1) 
        trace_hadamard = tf.expand_dims(tf.expand_dims(trace_hadamard, axis=-1), axis=-1)
        K_normalized = K_hadamard / trace_hadamard

        K_power = tf.linalg.matmul( K_normalized , K_normalized, grad_a=True, grad_b=True ) 
        trace_power = tf.reduce_sum(tf.linalg.diag_part(K_power), axis=-1)  
        H_alpha = (1 / (1 - self.alpha)) * tf.math.log(trace_power)
        return H_alpha

    def call(self, K_x, K_y_minus_1, K_y):
        
        K_x_exp = tf.expand_dims(K_x, axis=2)
        
        K_y_minus_1_exp = tf.expand_dims(K_y_minus_1, axis=1)
        K_y_exp = tf.expand_dims(K_y, axis=1)

        
        H_1 = self.compute_entropy(K_y_minus_1_exp * K_x_exp)
        H_2 = self.compute_entropy(K_y_exp * K_y_minus_1_exp * K_x_exp)
        H_3 = self.compute_entropy(K_y_exp * K_y_minus_1_exp)
        H_4 = self.compute_entropy(K_y_minus_1_exp)

        TE = H_1 - H_2 + H_3 - H_4
        
        return TE
        
@register_keras_serializable(package="CustomLayers")      
class RemoveDiagonalFlatten(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
    
        super().__init__(**kwargs)

    def call(self, inputs):
        
        shape_dyn = tf.shape(inputs)  
        batch_size = shape_dyn[0]
        c = tf.shape(inputs)[1]

        
        tf.debugging.assert_equal(
            tf.shape(inputs)[1], tf.shape(inputs)[2],
            message="RemoveDiagonalFlatten: la matriz de entrada no es cuadrada."
        )  
        diag_mask = tf.eye(c, dtype=inputs.dtype)  
        inputs_no_diag = inputs * (1 - diag_mask) 
        flattened = tf.reshape(inputs_no_diag, [batch_size, -1])  
        non_diag = tf.boolean_mask(flattened, tf.reshape(1 - diag_mask, [-1]), axis=1)
        num_features = c * (c - 1)  
        result = tf.reshape(non_diag, [batch_size, num_features])  

        return result

In [ ]:
# # Crear el modelo
# model = create_model()
# model.summary()

In [16]:
class Specificity(tf.keras.metrics.Metric):
    def __init__(self, name='specificity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
    def result(self):
        return self.tn.result() / (self.tn.result() + self.fp.result() + tf.keras.backend.epsilon())
    def reset_states(self):
        self.tn.reset_states()
        self.fp.reset_states()

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)
    def result(self):
        precision = self.tp.result() / (self.tp.result() + self.fp.result() + tf.keras.backend.epsilon())
        recall    = self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    def reset_states(self):
        self.tp.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

In [17]:
def create_model(dx, dy, tau, mu, kernel_type, length_scale, alpha):
    input_eeg = tf.keras.Input(shape=(22, 500), name='input_eeg')
    x = tf.keras.layers.DepthwiseConv1D(
        kernel_size=22,
        strides=1,
        padding='valid',
        activation='relu',
        depth_multiplier=1,
        data_format="channels_first",
        name='block1_depthwise_conv1d'
    )(input_eeg)
        
    
    x_block1 = tf.keras.layers.AveragePooling1D(
        pool_size=4, 
        strides=4, 
        padding='valid',
        data_format="channels_first", 
        name='block1_avg_pooling'
    )(x)

    # Bloque 2 (Takens)
    x_m, y_m1, y_t = TakensConv1D(dx=dx, dy=dy, tau=tau, mu=mu)(x_block1)
    x_m = tf.keras.layers.Dense(dx, activation=None, use_bias=False)(x_m)
    y_m1 = tf.keras.layers.Dense(dy, activation=None, use_bias=False)(y_m1)
    y_t = tf.keras.layers.Dense(1, activation=None, use_bias=False)(y_t)

    # KernelLayer con elección de tipo
    Kx = KernelLayer(kernel_type=kernel_type, length_scale=length_scale, alpha=alpha)(x_m)
    Ky1 = KernelLayer(kernel_type=kernel_type, length_scale=length_scale, alpha=alpha)(y_m1)
    Ky  = KernelLayer(kernel_type=kernel_type, length_scale=length_scale, alpha=alpha)(y_t)

    # Transfer Entropy
    TE = TransferEntropyLayer(alpha=2)(Kx, Ky1, Ky)
    x_flat = RemoveDiagonalFlatten()(TE)

    # Salida
    x_dense = tf.keras.layers.Dense(10, activation='relu')(x_flat)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(x_dense)

    model = tf.keras.Model(inputs=input_eeg, outputs=out)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            tf.keras.metrics.Recall(name='sensitivity'),
            Specificity(),
            F1Score()
        ]
    )
    return model

In [18]:
train_data, val_data, train_labels, val_labels = train_test_split(
    X, y,
    test_size=0.2,
    random_state=72,
    stratify=y
)

def build_model(hp):
    # 1) Lee los HP
    dx  = hp.Int('dx',  1, 10)
    dy  = hp.Int('dy',  1, 10)
    tau = hp.Int('tau', 1, 10)
    mu  = hp.Int('mu',  0, 10)

    # 2) Calcula el kernel_size que usará TakensConv1D
    kernel_size = mu + (dx - 1) * tau + 1

    # 3) Longitud temporal disponible tras el Bloque 1
    T_post_block1 = 109

    # 4) Rechaza combos que no encajen
    if kernel_size > T_post_block1:
        return None


    # Tipo de kernel
    kernel_type = hp.Choice('kernel_type', ['gaussian', 'rational_quadratic'])

    # Parámetros continuos
    length_scale = hp.Choice('length_scale', [10.0, 100.0, 1000.0])
    alpha        = hp.Float('alpha',         0.1, 2.0, sampling='log') if kernel_type=='rational_quadratic' else 0.5

    # Crear modelo con esos HP
    return create_model(dx, dy, tau, mu, kernel_type, length_scale, alpha)

# tunner
tuner = kt.BayesianOptimization(
    build_model,
    objective=kt.Objective("val_f1_score", direction="max"),
    max_trials=30, 
    executions_per_trial=2,
    directory="tuner_dir_2",
    project_name="takens_te_tuning_2"
)

# EarlyStopping por si no mejora la val_loss
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(
    x=train_data,
    y=train_labels,
    validation_data=(val_data, val_labels),
    epochs=50,
    callbacks=[stop_early]
)

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# 2) Obtén el Trial ganador desde el oracle
#    A) Si tu versión ofrece get_best_trials():
try:
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
except AttributeError:
    # B) Si no existe, usa la propiedad best_trials
    best_trial = tuner.oracle.best_trials[0]

# 3) Extrae la puntuación de val_f1_score
best_score = best_trial.score

# 4) Guarda todo en un DataFrame
import pandas as pd

best_dict = {
    'dx':            best_hp.get('dx'),
    'dy':            best_hp.get('dy'),
    'tau':           best_hp.get('tau'),
    'mu':            best_hp.get('mu'),
    'kernel_type':   best_hp.get('kernel_type'),
    'length_scale':  best_hp.get('length_scale'),
    'alpha':         best_hp.get('alpha'),
    'val_f1_score':  best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)
df_best.to_csv('best_hyperparameters_with_score.csv', index=False)

Trial 30 Complete [00h 01m 06s]
val_f1_score: 0.7128919064998627

Best val_f1_score So Far: 0.7507330477237701
Total elapsed time: 00h 32m 12s
   dx  dy  tau  mu         kernel_type  length_scale     alpha  val_f1_score
0   6   6    7   5  rational_quadratic        1000.0  0.313391      0.750733


In [13]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor

In [18]:
# # 1) Recolecta tus trials y best_hp (igual que antes)
# trials = [t for t in tuner.oracle.trials.values() if getattr(t, "score", None) is not None]
# X = np.array([[t.hyperparameters.get("length_scale"),
#                t.hyperparameters.get("alpha")]
#               for t in trials])
# y = np.array([t.score for t in trials])
# best_ls, best_alpha = best_hp.get("length_scale"), best_hp.get("alpha")

# # 2) Transforma length_scale a log10
# X_log = np.column_stack([np.log10(X[:,0]), X[:,1]])

# # 3) Kernel anisotrópico RBF
# kernel = RBF(length_scale=[1.0, 1.0],  # valores iniciales (se optimizan luego)
#              length_scale_bounds=[(1e-3, 1e3),  # ℓ en [1e-3,1e3]
#                                   (0.1, 2.0)])  # α en [0.1,2.0]
# gp = GaussianProcessRegressor(kernel=kernel,
#                               normalize_y=False)
# gp.fit(X_log, y)

# # 4) Crea rejilla muy ajustada (+/–10 %)
# ls_log_min, ls_log_max = X_log[:,0].min(), X_log[:,0].max()
# a_min,       a_max     = X_log[:,1].min(), X_log[:,1].max()
# delta_ls = 0.1*(ls_log_max - ls_log_min)
# delta_a  = 0.1*(a_max       - a_min)

# grid_ls_log = np.linspace(ls_log_min - delta_ls, ls_log_max + delta_ls, 100)
# grid_a      = np.linspace(a_min      - delta_a,   a_max      + delta_a, 100)
# Gx, Gy      = np.meshgrid(grid_ls_log, grid_a)
# XY_log      = np.vstack([Gx.ravel(), Gy.ravel()]).T

# # 5) Predições
# mean, std = gp.predict(XY_log, return_std=True)
# M = mean.reshape(Gx.shape)
# S = std.reshape(Gx.shape)

# # Niveles
# levels_m = np.linspace(M.min(), M.max(), 6)
# levels_s = np.linspace(S.min(), S.max(), 6)

# # 6) Dibuja
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,6), sharey=True)

# # –– Mean
# cf1 = ax1.contourf(10**Gx, Gy, M, levels=levels_m, cmap='viridis', extend='both')
# ax1.contour(10**Gx, Gy, M, levels=levels_m, colors='k', linewidths=0.7)
# ax1.scatter(10**X_log[:,0], X_log[:,1], c='white', edgecolors='k', s=40, label='Trials')
# ax1.scatter(best_ls, best_alpha, c='red', marker='X', s=100, label='Best HP')
# ax1.set_xscale('log'); ax1.set_title('Mean Prediction'); ax1.set_xlabel('length_scale'); ax1.set_ylabel('alpha')
# ax1.legend()
# cbar1 = fig.colorbar(cf1, ax=ax1, pad=0.02, label='F1 pred')
# cbar1.formatter.set_powerlimits((0,0)); cbar1.update_ticks()

# # –– Std. Dev.
# cf2 = ax2.contourf(10**Gx, Gy, S, levels=levels_s, cmap='viridis', extend='both')
# ax2.contour(10**Gx, Gy, S, levels=levels_s, colors='k', linewidths=0.7)
# ax2.scatter(10**X_log[:,0], X_log[:,1], c='white', edgecolors='k', s=40, label='Trials')
# ax2.scatter(best_ls, best_alpha, c='red', marker='X', s=100, label='Best HP')
# ax2.set_xscale('log'); ax2.set_title('Uncertainty (Std. Dev.)'); ax2.set_xlabel('length_scale')
# ax2.legend()
# cbar2 = fig.colorbar(cf2, ax=ax2, pad=0.02, label='Std. dev.')
# cbar2.formatter.set_powerlimits((0,0)); cbar2.update_ticks()

# plt.tight_layout()
# plt.show()

In [20]:
# 1) Recoge todos los trials completados
trials = [
    t for t in tuner.oracle.trials.values()
    if getattr(t, "score", None) is not None
]

# 2) Monta un DataFrame con las columnas clave
records = []
for t in trials:
    hp = t.hyperparameters
    records.append({
        'dx':            hp.get('dx'),
        'dy':            hp.get('dy'),
        'tau':           hp.get('tau'),
        'mu':            hp.get('mu'),
        'length_scale':  hp.get('length_scale'),
        # 'alpha':         hp.get('alpha'),
        'val_f1_score':  t.score
    })

df = pd.DataFrame(records)

# 3) Mira los 10 mejores
df_top10 = df.sort_values('val_f1_score', ascending=False).head(10)
print(df_top10)

    dx  dy  tau  mu  length_scale  val_f1_score
24   6   6    7   5        1000.0      0.750733
29   6   7    6   5        1000.0      0.712892
22   4   4    6   6         100.0      0.701351
0    4   5    1   1          10.0      0.694444
21   7   9    4   3        1000.0      0.684962
13   1   6    6   5         100.0      0.684685
28   6   5    7   6        1000.0      0.678523
26   6   5    5   4        1000.0      0.676933
3    6   2    6   3          10.0      0.675522
11   2   5    4   7        1000.0      0.674797


In [13]:
df.to_excel('all_trials.xlsx', index=False)
df_top10.to_excel('top10_trials.xlsx', index=False)